# Hyperparameter Tuning using HyperDrive

Import Dependencies. 

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import joblib
import os

## Dataset

This experiment uses a dataset from Kaggle regarding heart failures with 12 features to predict whether an event of death occurs or not. Therefore, this is a classification prediction task.

In [2]:
ws = Workspace.from_config()
experiment_name = 'experiment1'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

The parameters used in the HyperDrive configuration are as follows:
- early_termination_policy: Improving the computational efficiency by terminaly poorly perfoming runs.
- param_sampling: The parameter sampling method indicates the method of searching the hyperparameter space.

In [3]:
cpu_cluster_name = "computecluster"

# Verify if the clusters already exists
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing target, use it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2', max_nodes = 4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output = True)

Found existing target, use it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1)

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'--C':choice(1, 2, 3, 4),
'--max_iter':choice(16, 32, 64, 128)})

if "training" not in os.listdir():
    os.mkdir("./training")

# Create your estimator and hyperdrive config
est = SKLearn(source_directory='.',
entry_script='train.py',
compute_target=cpu_cluster)

hyperdrive_run_config = HyperDriveConfig(estimator=est,
hyperparameter_sampling = param_sampling,
primary_metric_name = 'Accuracy',
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
max_total_runs=25,
max_concurrent_runs=4,
policy=early_termination_policy)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [5]:
hyperdrive_run = experiment.submit(config = hyperdrive_run_config)

## Run Details

`RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d1a698d2-18d3-4000-9063-314dd7324c15
Web View: https://ml.azure.com/experiments/experiment1/runs/HD_d1a698d2-18d3-4000-9063-314dd7324c15?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-139017/workspaces/quick-starts-ws-139017

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-16T12:21:46.483091][API][INFO]Experiment created<END>\n""<START>[2021-02-16T12:21:46.961604][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-02-16T12:21:47.505551][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-02-16T12:21:48.0663703Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
#best_run_metrics = best_run.get_metrics()
print(best_run)

In [ ]:
best_run_metrics['Accuracy']

In [ ]:
model = best_run.register_model(model_name='best_hyperdrive_model', model_path = 'outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service